In [1]:
import pathlib
from dataset import Dataset, BAD_MOOD

DATA_DIR = pathlib.Path('data')
path = next(DATA_DIR.glob('*.csv'))
print('using file', path.name)

df = Dataset(csv_file_path=path)

using file daylio_export_2024_03_14.csv
Dataset(782 entries; last [1 hour 19 minutes 7 seconds ago]; mood: 3.799 ± 0.585)


# API

### `.head`

Use the `.head` method to look at the latest entries of the dataset

In [2]:
df.head()

Dataset(782 entries; last [1 hour 19 minutes 7 seconds ago]; mood: 3.799 ± 0.585)
[14.03.2024 22:24] 3.0 relaxed
[14.03.2024 17:20] 4.0 walking, shopping
[14.03.2024 14:48] 4.0 happy, home, cooking, satisfied
[14.03.2024 11:00] 3.0 unsure, worried, home
[14.03.2024 10:15] 4.0 Azat, home, coding, satisfied
...


In [3]:
df.head(2)

Dataset(782 entries; last [1 hour 19 minutes 7 seconds ago]; mood: 3.799 ± 0.585)
[14.03.2024 22:24] 3.0 relaxed
[14.03.2024 17:20] 4.0 walking, shopping
...


In [ ]:
# prints all entries
df.head(-1)

### `.sub`

Use the `.sub` method to filter entries and get a subset of the original dataset

- by included activities ('or' operator: all the entries which have at least one of the listed activities)

In [5]:
cycling_or_city_df = df.sub(include={'cycling', 'city'})
cycling_or_city_df.head()

Dataset(75 entries; last [19 days 10 hours 38 minutes 7 seconds ago]; mood: 4.173 ± 0.517)
[24.02.2024 13:05] 4.0 city, unsure, sad, grateful
[16.02.2024 19:30] 3.5 date, Lauren, city
[09.02.2024 23:06] 4.5 date, Lauren, grateful, city, happy, satisfied
[27.01.2024 18:47] 4.0 city, tired, satisfied
[27.01.2024 16:03] 4.0 bored, city, tired, satisfied
...


- by excluded activities (entries which don't have any of the listed activities)

_for both `include` and `exclude` the argument can be either `str` or `set[str]`_

In [6]:
without_friends = df.sub(exclude='friends')
without_friends.head()

Dataset(679 entries; last [1 hour 19 minutes 7 seconds ago]; mood: 3.771 ± 0.583)
[14.03.2024 22:24] 3.0 relaxed
[14.03.2024 17:20] 4.0 walking, shopping
[14.03.2024 14:48] 4.0 happy, home, cooking, satisfied
[14.03.2024 11:00] 3.0 unsure, worried, home
[14.03.2024 10:15] 4.0 Azat, home, coding, satisfied
...


- by mood values (can be either a value or a Container of values)

In [7]:
bad_mood = df.sub(mood=BAD_MOOD)
bad_mood

Dataset(33 entries; last [11 days 13 hours 23 minutes 7 seconds ago]; mood: 2.273 ± 0.253)

- by included activities ('and' operator: all entries which have all the listed activities)

_Note: this is achieved by subscribing the dataset twice; since .sub method returns a new dataset, it can be done in one line._

In [8]:
cycling_and_swimming = df.sub(include='cycling').sub(include='swimming')
cycling_and_swimming.head()
print(cycling_and_swimming.activities().most_common(7))

Dataset(10 entries; last [5 months 1 day 1 hour 33 minutes 7 seconds ago]; mood: 4.850 ± 0.474)
[13.10.2023 20:10] 5.0 cycling, tired, swimming
[16.09.2023 20:58] 5.0 cycling, tired, relaxed, satisfied, swimming
[04.09.2023 20:44] 5.0 Damir, swimming, cycling, tired, relaxed, satisfied
[22.08.2023 20:27] 5.0 cycling, tired, friends, satisfied, swimming
[19.08.2023 20:32] 5.0 Damir, cycling, tired, relaxed, friends, swimming
...
[('cycling', 10), ('tired', 10), ('swimming', 10), ('satisfied', 7), ('relaxed', 6), ('friends', 6), ('Damir', 5)]


In [9]:
df.sub(include='cycling', exclude='swimming')

Dataset(24 entries; last [1 month 18 days 17 hours 40 minutes 7 seconds ago]; mood: 4.167 ± 0.458)

- by a subtring (or substrings) in the `note`

In [10]:
df_ktane = df.sub(note_contains='ktane')
df_ktane.head(-1)

Dataset(8 entries; last [6 days 6 hours 17 minutes 7 seconds ago]; mood: 4.000 ± 0.463)
[08.03.2024 17:26] 4.0 amused, gaming, annoyed, Leha, home, happy, friends, satisfied
[22.11.2023 21:41] 4.0 amused, gaming, home, Leha, movies & series, Martina, satisfied
[10.10.2023 22:39] 3.5 amused, gaming, Leha, productive, stressed, friends, home
[06.10.2023 20:45] 4.0 gaming, Leha, happy, friends, excited
[04.10.2023 20:13] 5.0 new place, movies & series, Lauren, grateful, happy, friends, satisfied
[07.07.2023 20:00] 3.5 coding, gaming, unsure, Leha, shopping, dota, walking, study, friends
[03.07.2023 20:05] 4.0 coding, gaming, Leha, tired, happy, stressed, friends, excited, home
[01.07.2023 20:00] 4.0 gaming, relaxed, movies & series, Leha, Martina, happy, friends, excited, home


- by a predicate function (which takes an entry as an argument and returns a boolean)

In [11]:
from dataset import Entry


def pred(entry: Entry) -> bool:
    """It is sunday, the mood > 4, the note is not empty"""
    return entry.full_date.weekday() == 6 and entry.mood > 4 and entry.note != ''


df_sunday_good_mood_has_note = df.sub(predicate=pred)
df_sunday_good_mood_has_note

Dataset(23 entries; last [18 days 4 hours 41 minutes 7 seconds ago]; mood: 4.739 ± 0.255)

### `.mood` and `.mood_std`

Use the `.mood` method to get the average mood of all the entries in the dataset and the `.mood_std` method to get the standard deviation of the mood values.

These values are also shown when calling `.head()`.

In [12]:
print(
    round(cycling_or_city_df.mood(), 3),
    round(cycling_or_city_df.mood_std(), 3)
)

4.173 0.517


### `.activities`

Use the `.activities` method to get a Counter object of all activities in the dataset

In [13]:
cnt = df.activities()
print(f'most common: {cnt.most_common(3)}')
print(f'least common: {cnt.most_common()[-1:-6:-1]}')

most common: [('home', 459), ('relaxed', 244), ('satisfied', 244)]
least common: [('guitar', 2), ('homework', 2), ('Dad', 2), ('snacking', 3), ('cinema', 3)]


### `.get_datetimes`

Use the `.get_datetimes` method to get the list of all points in time when an entry was created. The values are sorted in descending order by the date of creation.

In [14]:
df.get_datetimes()[:5]

[datetime.datetime(2024, 3, 14, 22, 24),
 datetime.datetime(2024, 3, 14, 17, 20),
 datetime.datetime(2024, 3, 14, 14, 48),
 datetime.datetime(2024, 3, 14, 11, 0),
 datetime.datetime(2024, 3, 14, 10, 15)]

### `.group_by`
Use the `.group_by` method to get a dictionary mapping groups to the list of entries in that group.

The groups are one of `['day', 'month']` and the entries are sorted in ascending order.

Note: this method (like many others) uses lru_cache and since the Dataset is weakly-immutable, it is safe to use it.

In [15]:
groups = df.group_by('day')
for day, entries in groups.items():
    print(f'{day}: {len(entries)}')
    break

2023-07-01: 1


In [16]:
groups = df.group_by('month')
for day, entries in groups.items():
    print(f'{day}: {len(entries)}')
    break

2023-07-01: 95


### `.stats`

Use the `.stats` method to get a custom StatsResult object which contains the following information:

In [17]:
help(df.stats)

Help on method stats in module dataset:

stats() -> utils.StatsResult method of dataset.Dataset instance
    Returns the following statistics:
        - mood (avg ± std)
        - note length [num symbols] (avg ± std)
        - entries frequency [entries per day] (median)
    as a StatsResult object.



In [18]:
df.sub(include='home').stats()

Mood: 3.744 ± 0.526
Note length: 38.307 ± 62.366 symbols
Entries frequency: 3.061 entries per day (once every 7 hours 50 minutes 29 seconds)

### `.__iter__`
`Dataset` defines `__iter__` method, so it can be used in `for` loops. The entries are sorted in descending order by the date of creation.

In [19]:
for entry in df:
    print(entry)
    break
# or
df_iter = iter(df)
print(next(df_iter), next(df_iter), sep='\n')

[14.03.2024 22:24] 3.0 relaxed
[14.03.2024 22:24] 3.0 relaxed
[14.03.2024 17:20] 4.0 walking, shopping


### `.__getitem__`
`Dataset` defines `__getitem__` method where the argument is 
- a date in a string format (e.g. `'01.01.2024'`), returns a new Dataset with all entries on that date;
- a slice object (e.g. `slice('01.01.2024', '01.01.2025')`), returns a new Dataset with all entries which were created between the two dates (the "stop" date is not included);

In [20]:
july22 = df['22.07.2023']

In [21]:
july22

Dataset(6 entries; last [7 months 23 days 3 hours 26 minutes 7 seconds ago]; mood: 3.583 ± 0.376)

- by a particular date period (by using slices)

In [22]:
ny_eve = df['29.12.2023':'03.01.2024']
ny_eve

Dataset(16 entries; last [2 months 11 days 3 hours 48 minutes 7 seconds ago]; mood: 3.969 ± 0.499)

In [23]:
end_of_november_2023 = df['29.11.2023':'01.12.2023']
end_of_november_2023.head()

Dataset(5 entries; last [3 months 13 days 18 hours 6 minutes 7 seconds ago]; mood: 3.700 ± 0.274)
[30.11.2023 23:37] 3.5 date, Lauren, tired, city
[30.11.2023 17:06] 4.0 coding, home, excited, satisfied
[30.11.2023 11:07] 3.5 unsure, coding, Leha, home
[29.11.2023 21:20] 3.5 amused, gaming, Azat, Leha, social
[29.11.2023 16:42] 4.0 coding, happy, satisfied, excited, home


In [24]:
before_aug_2023 = df[:'01.08.2023']
before_aug_2023.head()

Dataset(95 entries; last [7 months 14 days 6 hours 49 minutes 8 seconds ago]; mood: 3.589 ± 0.688)
[31.07.2023 18:54] 3.5 relaxed, cleaning
[31.07.2023 17:22] 3.0 unsure, study, tired, home
[31.07.2023 16:45] 4.0 study, relaxed, satisfied, cooking, home
[31.07.2023 12:00] 3.5 bored, procrastinating, unsure, relaxed, home
[30.07.2023 23:26] 4.0 amused, Lauren, grateful, laundry, happy, friends, home
...


### `__call__`
Return a list of entries for a particular day. The argument is a date in a string format (e.g. `'01.01.2024'`).

In [25]:
ny_entries = df('31.12.2023')
ny_entries

[[31.12.2023 16:48] 4.5 date, movies & series, Lauren, cooking, happy, excited, home,
 [31.12.2023 16:48] 2.5 date, annoyed, Lauren, photography, city, sad,
 [31.12.2023 21:51] 3.5 date, Lauren, movies & series, satisfied, cooking, home,
 [31.12.2023 22:21] 4.0 family, Dad, Mom, happy, nervous, home,
 [31.12.2023 23:57] 4.0 happy, date, Lauren]

### `@ <datetime_string>` (or `.at`)
Returns a single entry created at a particular datetime or `None` if there are no entries created at that datetime.

Input: a datetime in a string format (e.g. `'01.01.2024 12:00'`).

In [26]:
# or
entry = df @ '22.07.2023 17:09' # or df.at(...)
entry

[22.07.2023 17:09] 3.5 relaxed, cooking, study, home

# Analysis examples

## Mood analysis

In [27]:
activity = 'cycling'
mood_with, mood_without = df.mood_with_without(activity)

In [28]:
print(f'''"{activity}"
with: {mood_with:.2f}
without: {mood_without:.2f}
change: {(mood_with - mood_without)/mood_without:.1%}'''
)

"cycling"
with: 4.37
without: 3.77
change: 15.8%


## Complete analysis

In [29]:
complete_analysis = df.complete_analysis()

In [30]:
print(f'analysed {len(complete_analysis)} actvities')
for _name, _with, _without, _change, _num_occ in complete_analysis[:8] + complete_analysis[-8:]:
    print(f'[{_name:^15}]: {_change:.1%} (with: {_with:.2f}, without: {_without:.2f}); occured {_num_occ} times')

analysed 52 actvities
[     happy     ]: 19.2% (with: 4.35, without: 3.65); occured 165 times
[   swimming    ]: 17.3% (with: 4.43, without: 3.78); occured 28 times
[    cycling    ]: 15.8% (with: 4.37, without: 3.77); occured 34 times
[   grateful    ]: 15.5% (with: 4.32, without: 3.74); occured 82 times
[     Damir     ]: 14.4% (with: 4.33, without: 3.78); occured 23 times
[   satisfied   ]: 11.5% (with: 4.09, without: 3.67); occured 244 times
[    Lauren     ]: 10.4% (with: 4.10, without: 3.71); occured 171 times
[     city      ]: 9.9% (with: 4.15, without: 3.77); occured 50 times
[     exam      ]: -13.1% (with: 3.31, without: 3.81); occured 13 times
[    worried    ]: -17.9% (with: 3.14, without: 3.82); occured 29 times
[   stressed    ]: -18.0% (with: 3.12, without: 3.81); occured 12 times
[     sick      ]: -19.9% (with: 3.06, without: 3.82); occured 18 times
[      sad      ]: -21.0% (with: 3.02, without: 3.82); occured 24 times
[    annoyed    ]: -21.4% (with: 3.00, without: 

## Mood plot

In [31]:
df.mood_plot('day')

## Bar plot by hour/day/month

In [32]:
df.by_time_bar_plot('hour')

In [33]:
df.by_time_bar_plot('day')

In [34]:
df.by_time_bar_plot('weekday')

In [35]:
df.by_time_bar_plot('month')

## Other features

In [36]:
df.note_length_plot(400)

TypeError: Dataset.group_by() missing 1 required positional argument: 'what'

# Making a network

In [ ]:
from itertools import combinations
from collections import Counter

from pyvis.network import Network


def color_gradient(start: str, finish: str, t: float) -> str:
    """Return a color that is a linear interpolation between start and finish"""
    start_rgb = int(start[1:3], 16), int(start[3:5], 16), int(start[5:7], 16)
    finish_rgb = int(finish[1:3], 16), int(finish[3:5], 16), int(finish[5:7], 16)
    r = int(start_rgb[0] + t * (finish_rgb[0] - start_rgb[0]))
    g = int(start_rgb[1] + t * (finish_rgb[1] - start_rgb[1]))
    b = int(start_rgb[2] + t * (finish_rgb[2] - start_rgb[2]))
    return f'#{r:02x}{g:02x}{b:02x}'


def network_from_df(df: Dataset) -> Network:
    net = Network(height='1000px', width='100%', directed=False, bgcolor='#222222', font_color=True)
    activities_counter = df.activities()
    most_common = activities_counter.most_common(20)
    complete_analysis = df.complete_analysis()
    activity_to_change = {ca.activity: ca.change for ca in complete_analysis}
    for activity, count in most_common:
        net.add_node(activity, label=activity, value=count, title=f'occured {count} times;\nmood change: {activity_to_change[activity]:.1%}')
    common_activities = set(activity for activity, _ in most_common)
    all_edges = []
    for entry in df:
        for pair in combinations(entry.activities, 2):
            if pair[0] not in common_activities or pair[1] not in common_activities:
                continue
            all_edges.append(tuple(sorted(pair)))
    edges_counter = Counter(all_edges)
    for (source, target), count in edges_counter.items():
        net.add_edge(source, target, value=count)
    return net

In [ ]:
net = network_from_df(df)
net.force_atlas_2based(spring_strength=0.02, gravity=-30, damping=0.2)
net.show('network.html', notebook=False)